In [1]:
!pip install onnxruntime-gpu -q

import torch
import torch.nn as nn
from torch.utils.data import DataLoader, Subset
from torchvision import datasets, transforms
import torchvision.transforms as T
import numpy as np
from scipy.stats import ks_2samp
import onnxruntime as ort
import os
from tqdm import tqdm

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Device: {device}")

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 252.6/252.6 MB 5.2 MB/s eta 0:00:00:00:0100:01
Device: cpu


# Configuration

In [2]:
path = "/kaggle/input/datasets/paultimothymooney/chest-xray-pneumonia"
BATCH_SIZE = 64
ALPHA = 0.05  # Seuil pour le test KS

checkpoint = torch.load("/kaggle/input/datasets/asthehis/baseline-pneumonia-model/baseline_metrics.pt")
MEAN = checkpoint['mean']
STD = checkpoint['std']
print(f"Normalisation - MEAN: {MEAN}, STD: {STD}")

Normalisation - MEAN: [0.4823, 0.4823, 0.4823], STD: [0.2216, 0.2216, 0.2216]


# Chargement du modèle

In [3]:
onnx_path = "/kaggle/input/models/asthehis/onnx-model/onnx/default/1/model_optimized.onnx"

ort_session = ort.InferenceSession(
    onnx_path,
    providers=['CUDAExecutionProvider', 'CPUExecutionProvider']
)

input_name = ort_session.get_inputs()[0].name
output_name = ort_session.get_outputs()[0].name
print(f"✅ Modèle ONNX chargé")
print(f"   - Input: {input_name}")
print(f"   - Output: {output_name}")

*************** EP Error ***************
EP Error /onnxruntime_src/onnxruntime/core/providers/cuda/cuda_call.cc:129 std::conditional_t<THRW, void, onnxruntime::common::Status> onnxruntime::CudaCall(ERRTYPE, const char*, const char*, SUCCTYPE, const char*, const char*, int) [with ERRTYPE = cudaError; bool THRW = true; SUCCTYPE = cudaError; std::conditional_t<THRW, void, common::Status> = void] /onnxruntime_src/onnxruntime/core/providers/cuda/cuda_call.cc:121 std::conditional_t<THRW, void, onnxruntime::common::Status> onnxruntime::CudaCall(ERRTYPE, const char*, const char*, SUCCTYPE, const char*, const char*, int) [with ERRTYPE = cudaError; bool THRW = true; SUCCTYPE = cudaError; std::conditional_t<THRW, void, common::Status> = void] CUDA failure 35: CUDA driver version is insufficient for CUDA runtime version ; GPU=-1 ; hostname=266fc802359e ; file=/onnxruntime_src/onnxruntime/core/providers/cuda/cuda_execution_provider.cc ; line=334 ; expr=cudaSetDevice(info_.device_id); 

 when using 

# Préparation des données

In [12]:
# Calcul des moyennes pour 1 seul canal
MEAN_GRAY = [sum(MEAN) / 3]
STD_GRAY = [sum(STD) / 3]

# La transformation de test doit impérativement être en Grayscale(1)
test_transform_gray = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.Grayscale(num_output_channels=1), 
    transforms.ToTensor(),
    transforms.Normalize(mean=MEAN_GRAY, std=STD_GRAY)
])

In [13]:
test_transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=MEAN, std=STD)
])

# Charger le dataset de test
base_path = os.path.join(path, "chest_xray")
test_dataset = datasets.ImageFolder(
    os.path.join(base_path, "test"), 
    transform=test_transform_gray
)

print(f"📊 Dataset de test : {len(test_dataset)} images")
print(f"   Classes : {test_dataset.class_to_idx}")

# Diviser en 2 groupes : CLEAN (50%) et DRIFT (50%)
total_test = len(test_dataset)
split_point = total_test // 2

# Mélanger de façon reproductible
torch.manual_seed(42)
indices = torch.randperm(total_test).tolist()

clean_indices = indices[:split_point]
drift_indices = indices[split_point:]

# Créer les subsets
clean_subset = Subset(test_dataset, clean_indices)
drift_subset = Subset(test_dataset, drift_indices)

# DataLoaders
clean_loader = DataLoader(
    clean_subset, 
    batch_size=BATCH_SIZE, 
    shuffle=False,
    num_workers=2,
    pin_memory=True
)

drift_loader = DataLoader(
    drift_subset, 
    batch_size=BATCH_SIZE, 
    shuffle=False,
    num_workers=2,
    pin_memory=True
)

print(f"✅ Données divisées :")
print(f"   - CLEAN: {len(clean_subset)} images")
print(f"   - DRIFT: {len(drift_subset)} images")

📊 Dataset de test : 624 images
   Classes : {'NORMAL': 0, 'PNEUMONIA': 1}
✅ Données divisées :
   - CLEAN: 312 images
   - DRIFT: 312 images


# Simulation de Drift

In [14]:
drift_transforms = T.Compose([
    T.GaussianBlur(kernel_size=15, sigma=(5.0, 7.0)),  # Flou important
    T.Lambda(lambda x: torch.clamp(x + torch.randn_like(x) * 0.15, 0, 1))  # Bruit + clamp
])

print("✅ Transformations de drift définies (flou + bruit)")

✅ Transformations de drift définies (flou + bruit)


# Fonction d'inférence

In [7]:
def predict_onnx(ort_session, images, input_name):
    """
    Prédictions avec ONNX Runtime
    
    Args:
        ort_session: Session ONNX
        images: Tensor PyTorch (B, C, H, W)
        input_name: Nom de l'input ONNX
    
    Returns:
        probs: Probabilités (numpy array)
        preds: Prédictions (numpy array)
    """
    # Convertir en numpy float32
    ort_inputs = {input_name: images.cpu().numpy()}
    
    # Inférence ONNX
    ort_outputs = ort_session.run(None, ort_inputs)
    
    # ort_outputs[0] contient les logits
    logits = ort_outputs[0]
    
    # Calculer les probabilités avec softmax
    exp_logits = np.exp(logits - np.max(logits, axis=1, keepdims=True))
    probs = exp_logits / np.sum(exp_logits, axis=1, keepdims=True)
    
    # Prédictions
    preds = np.argmax(logits, axis=1)
    
    return probs, preds

# Évaluation avec et sans Drift

In [8]:
def evaluate_with_onnx(ort_session, loader, input_name, apply_drift=False, desc="Evaluation"):
    """
    Évalue le modèle ONNX sur un loader
    
    Args:
        ort_session: Session ONNX
        loader: DataLoader
        input_name: Nom de l'input ONNX
        apply_drift: Appliquer les transformations de drift
        desc: Description pour la barre de progression
    
    Returns:
        accuracy: Précision
        all_probs: Toutes les probabilités pour la classe 1
        all_preds: Toutes les prédictions
        all_labels: Toutes les vraies étiquettes
    """
    all_preds = []
    all_labels = []
    all_probs = []
    
    with torch.no_grad():
        for images, labels in tqdm(loader, desc=desc):
            # Appliquer le drift si demandé
            if apply_drift:
                images = drift_transforms(images)
            
            # Prédictions ONNX
            probs, preds = predict_onnx(ort_session, images, input_name)
            
            # Probabilités pour la classe PNEUMONIA (index 1)
            probs_pneumonia = probs[:, 1]
            
            all_preds.extend(preds)
            all_labels.extend(labels.numpy())
            all_probs.extend(probs_pneumonia)
    
    # Calcul de l'accuracy
    all_preds = np.array(all_preds)
    all_labels = np.array(all_labels)
    all_probs = np.array(all_probs)
    
    accuracy = (all_preds == all_labels).mean()
    
    return accuracy, all_probs, all_preds, all_labels

# Évaluation sur données clean

In [15]:
# Vérifier ton modèle PyTorch original
import torchvision.models as models
import torch.nn as nn

def get_model_for_drift():
    resnet = models.resnet18(weights=None)
    num_ftrs = resnet.fc.in_features
    resnet.fc = nn.Sequential(
        nn.Linear(num_ftrs, 256),
        nn.ReLU(),
        nn.Dropout(0.4),
        nn.Linear(256, 2)
    )
    return resnet

model = get_model_for_drift()

# TEST : Combien de canaux attend le modèle ?
print(f"Premier layer du ResNet: {model.conv1}")
print(f"Nombre de canaux d'entrée: {model.conv1.in_channels}")

Premier layer du ResNet: Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
Nombre de canaux d'entrée: 3


In [16]:
print("🔍 DIAGNOSTIC DU MODÈLE ONNX")
print("="*60)

# Informations sur l'input
input_info = ort_session.get_inputs()[0]
print(f"Input Name: {input_info.name}")
print(f"Input Shape: {input_info.shape}")
print(f"Input Type: {input_info.type}")

# Informations sur l'output
output_info = ort_session.get_outputs()[0]
print(f"\nOutput Name: {output_info.name}")
print(f"Output Shape: {output_info.shape}")
print(f"Output Type: {output_info.type}")

print("\n" + "="*60)

🔍 DIAGNOSTIC DU MODÈLE ONNX
Input Name: input
Input Shape: ['batch_size', 1, 224, 224]
Input Type: tensor(float)

Output Name: logits
Output Shape: ['batch_size', 2]
Output Type: tensor(float)



In [17]:
print("\n" + "="*60)
print("📊 ÉVALUATION SUR DONNÉES CLEAN (sans drift)")
print("="*60)

acc_clean, probs_clean, preds_clean, labels_clean = evaluate_with_onnx(
    ort_session, 
    clean_loader, 
    input_name, 
    apply_drift=False,
    desc="Clean Data"
)

print(f"\n✅ Résultats CLEAN:")
print(f"   - Accuracy: {acc_clean:.2%}")
print(f"   - Confiance moyenne: {probs_clean.mean():.3f}")
print(f"   - Confiance médiane: {np.median(probs_clean):.3f}")


📊 ÉVALUATION SUR DONNÉES CLEAN (sans drift)


Clean Data: 100%|██████████| 5/5 [00:12<00:00,  2.45s/it]


✅ Résultats CLEAN:
   - Accuracy: 88.78%
   - Confiance moyenne: 0.588
   - Confiance médiane: 0.781


# Évaluation des données drift

In [18]:
print("\n" + "="*60)
print("🌊 ÉVALUATION SUR DONNÉES DRIFTÉES (avec flou + bruit)")
print("="*60)

acc_drift, probs_drift, preds_drift, labels_drift = evaluate_with_onnx(
    ort_session, 
    drift_loader, 
    input_name, 
    apply_drift=True,
    desc="Drifted Data"
)

print(f"\n✅ Résultats DRIFT:")
print(f"   - Accuracy: {acc_drift:.2%}")
print(f"   - Confiance moyenne: {probs_drift.mean():.3f}")
print(f"   - Confiance médiane: {np.median(probs_drift):.3f}")


🌊 ÉVALUATION SUR DONNÉES DRIFTÉES (avec flou + bruit)


Drifted Data: 100%|██████████| 5/5 [00:12<00:00,  2.46s/it]


✅ Résultats DRIFT:
   - Accuracy: 63.14%
   - Confiance moyenne: 0.984
   - Confiance médiane: 0.998


# Comparaison des performances

In [19]:
print("\n" + "="*60)
print("📉 IMPACT DU DRIFT SUR LES PERFORMANCES")
print("="*60)

performance_drop = acc_clean - acc_drift
confidence_drop = probs_clean.mean() - probs_drift.mean()

print(f"\n🔴 Performance Drop:")
print(f"   - Accuracy: {performance_drop:.2%} (de {acc_clean:.2%} à {acc_drift:.2%})")
print(f"   - Confiance: {confidence_drop:.3f} (de {probs_clean.mean():.3f} à {probs_drift.mean():.3f})")


📉 IMPACT DU DRIFT SUR LES PERFORMANCES

🔴 Performance Drop:
   - Accuracy: 25.64% (de 88.78% à 63.14%)
   - Confiance: -0.396 (de 0.588 à 0.984)


# Test KS

In [20]:
print("\n" + "="*60)
print("📊 TEST STATISTIQUE KS (Kolmogorov-Smirnov)")
print("="*60)

# Test KS sur les distributions de probabilités
ks_stat, p_value = ks_2samp(probs_clean, probs_drift)

print(f"\n✅ Résultats du test KS:")
print(f"   - Statistique KS: {ks_stat:.4f}")
print(f"   - P-value: {p_value:.4e}")
print(f"   - Seuil alpha: {ALPHA}")


📊 TEST STATISTIQUE KS (Kolmogorov-Smirnov)

✅ Résultats du test KS:
   - Statistique KS: 0.5609
   - P-value: 1.5069e-45
   - Seuil alpha: 0.05


# Détection de retraining

In [21]:
print("\n" + "="*60)
print("🎯 DÉCISION MLOPS")
print("="*60)

drift_detected = p_value < ALPHA

if drift_detected:
    print("\n🚨 DRIFT DÉTECTÉ !")
    print(f"   ✅ P-value ({p_value:.4e}) < Seuil ({ALPHA})")
    print(f"   ✅ Performance drop: {performance_drop:.2%}")
    print("\n🔄 DÉCISION: RETRAINING AUTOMATIQUE DÉCLENCHÉ")
    print("   Raisons:")
    print(f"   1. Dérive statistiquement significative (KS test)")
    print(f"   2. Dégradation de performance > 10%")
    print(f"   3. Baisse de confiance des prédictions")
else:
    print("\n✅ AUCUN DRIFT DÉTECTÉ")
    print(f"   P-value ({p_value:.4e}) ≥ Seuil ({ALPHA})")
    print("\n📊 DÉCISION: MODÈLE STABLE, PAS DE RETRAINING NÉCESSAIRE")


🎯 DÉCISION MLOPS

🚨 DRIFT DÉTECTÉ !
   ✅ P-value (1.5069e-45) < Seuil (0.05)
   ✅ Performance drop: 25.64%

🔄 DÉCISION: RETRAINING AUTOMATIQUE DÉCLENCHÉ
   Raisons:
   1. Dérive statistiquement significative (KS test)
   2. Dégradation de performance > 10%
   3. Baisse de confiance des prédictions


# Distance MMD

In [22]:
print("\n" + "="*60)
print("🧮 CALCUL DE LA DISTANCE MMD (Embedding Drift)")
print("="*60)

def compute_mmd(x, y, kernel_bandwidth=1.0):
    """
    Calcul de la Maximum Mean Discrepancy (MMD)
    
    Args:
        x: Array numpy (n_samples,)
        y: Array numpy (n_samples,)
        kernel_bandwidth: Paramètre du noyau RBF
    
    Returns:
        mmd: Distance MMD
    """
    # Convertir en tensors PyTorch
    x = torch.tensor(x, dtype=torch.float32).unsqueeze(1)
    y = torch.tensor(y, dtype=torch.float32).unsqueeze(1)
    
    # Calcul des distances
    xx = torch.mm(x, x.t())
    yy = torch.mm(y, y.t())
    xy = torch.mm(x, y.t())
    
    rx = xx.diag().unsqueeze(0).expand_as(xx)
    ry = yy.diag().unsqueeze(0).expand_as(yy)
    
    dxx = rx.t() + rx - 2. * xx
    dyy = ry.t() + ry - 2. * yy
    dxy = rx.t() + ry - 2. * xy
    
    # Noyau RBF (Gaussian)
    XX = torch.exp(-0.5 * dxx / kernel_bandwidth).mean()
    YY = torch.exp(-0.5 * dyy / kernel_bandwidth).mean()
    XY = torch.exp(-0.5 * dxy / kernel_bandwidth).mean()
    
    mmd = XX + YY - 2. * XY
    
    return mmd.item()

mmd_distance = compute_mmd(probs_clean, probs_drift)

print(f"\n✅ Distance MMD: {mmd_distance:.4f}")
print(f"   Interprétation:")
if mmd_distance > 0.1:
    print(f"   🔴 FORTE dérive (MMD > 0.1)")
elif mmd_distance > 0.05:
    print(f"   🟠 MOYENNE dérive (0.05 < MMD < 0.1)")
else:
    print(f"   🟢 FAIBLE dérive (MMD < 0.05)")


🧮 CALCUL DE LA DISTANCE MMD (Embedding Drift)

✅ Distance MMD: 0.1261
   Interprétation:
   🔴 FORTE dérive (MMD > 0.1)


# Sauvegarde des résultats

In [23]:
print("\n" + "="*60)
print("💾 SAUVEGARDE DES RÉSULTATS")
print("="*60)

drift_results = {
    'clean_accuracy': acc_clean,
    'drift_accuracy': acc_drift,
    'performance_drop': performance_drop,
    'ks_statistic': ks_stat,
    'ks_pvalue': p_value,
    'mmd_distance': mmd_distance,
    'drift_detected': drift_detected,
    'clean_probs': probs_clean,
    'drift_probs': probs_drift,
    'threshold_alpha': ALPHA
}

torch.save(drift_results, 'drift_detection_results.pt')
print("✅ Résultats sauvegardés dans 'drift_detection_results.pt'")


💾 SAUVEGARDE DES RÉSULTATS
✅ Résultats sauvegardés dans 'drift_detection_results.pt'
